In [1]:
import os
import tensorflow as tf
import numpy as np 

/home/aditya/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
os.chdir('/home/aditya/Downloads/TensorflowDatasets/train')

### Extracting Filenames and Classes

In [3]:
classes=[]
images=[]
for i in os.listdir('/home/aditya/Downloads/TensorflowDatasets/train'):
    filename=tf.read_file(i)
    images.append(filename)
    if i[0]=='c':
        classes.append(0)
    else:
        classes.append(1)

### One_hot encoding of Classes

In [4]:
y_true=tf.one_hot(classes,2)

In [5]:
images[1]

<tf.Tensor 'ReadFile_1:0' shape=() dtype=string>

### Creating Training(24000 images) and Validation Datasets(1000 images)

In [6]:
dataset = tf.data.Dataset.from_tensor_slices((images[:24000], y_true[:24000]))
validation_dataset = tf.data.Dataset.from_tensor_slices((images[24000:25000], y_true[24000:25000]))

In [7]:
len(images)

25000

In [8]:
dataset

<TensorSliceDataset shapes: ((), (2,)), types: (tf.string, tf.float32)>

### Getting Images from Filenames

In [9]:
def parse_function(images, y_true):
    image = tf.image.decode_jpeg(images, channels=3)
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize_images(image, [64, 64])
    return image, y_true

In [10]:
dataset = dataset.map(parse_function, num_parallel_calls=4)
validation_dataset = validation_dataset.map(parse_function, num_parallel_calls=4)

### Augmentation(Flip_up_down,Flip_Left_Right,Brightness,Saturation) with 50% chance. 

In [11]:
def train_preprocess(image, y_true):
    a=np.random.randint(1,5)
    b=np.random.randint(1,3)
    if b==1:
        image=image
    else:
        if a==1:
            image = tf.image.random_flip_left_right(image)
        elif a==2:
            image = tf.image.random_brightness(image, max_delta=32.0 / 255.0)
        elif a==3:
            image = tf.image.random_saturation(image, lower=0.5, upper=1.5)
        else:
            image = tf.image.random_flip_up_down(image)
    image = tf.clip_by_value(image, 0.0, 1.0)
    return image,y_true

In [12]:
dataset = dataset.map(train_preprocess, num_parallel_calls=2)

In [13]:
dataset

<ParallelMapDataset shapes: ((64, 64, 3), (2,)), types: (tf.float32, tf.float32)>

### Shuffling Dataset and setting epochs=5

In [14]:
dataset=dataset.shuffle(24000)
dataset=dataset.repeat(5)

### Batchsize=16

In [15]:
dataset=dataset.batch(16)

In [16]:
dataset

<BatchDataset shapes: ((?, 64, 64, 3), (?, 2)), types: (tf.float32, tf.float32)>

### Reinitializable Iterator

In [17]:
iterator = tf.data.Iterator.from_structure(dataset.output_types,
                                           dataset.output_shapes)
X,y = iterator.get_next()

In [18]:
y

<tf.Tensor 'IteratorGetNext:1' shape=(?, 2) dtype=float32>

In [19]:
os.getcwd()

'/home/aditya/Downloads/TensorflowDatasets/train'

In [20]:
dataset

<BatchDataset shapes: ((?, 64, 64, 3), (?, 2)), types: (tf.float32, tf.float32)>

In [21]:
X=tf.reshape(X,[-1,64,64,3])

### VGG 1st Layer + 1 Fully Connected into Prediction

In [22]:
conv1=tf.layers.conv2d(X,64,3)
conv2=tf.layers.conv2d(conv1,64,3)
maxpool1=tf.layers.max_pooling2d(conv2,[2,2],[2,2])
flat=tf.layers.flatten(maxpool1)
dense1=tf.layers.dense(flat,1024)
y_pred=tf.layers.dense(dense1,2)

In [23]:
y_pred

<tf.Tensor 'dense_1/BiasAdd:0' shape=(?, 2) dtype=float32>

In [24]:
y_true

<tf.Tensor 'one_hot:0' shape=(25000, 2) dtype=float32>

### Initializing Iterator

In [25]:
validation_dataset=validation_dataset.batch(16)
training_init_op = iterator.make_initializer(dataset)
validation_init_op = iterator.make_initializer(validation_dataset)

### Training and Loss(Softmax)

In [26]:
loss_op=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=y_pred))
train_op=tf.train.AdamOptimizer().minimize(loss_op) 

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



### Accuracy 

In [27]:
prediction=tf.equal(tf.argmax(y_pred,1),tf.argmax(y,1))
accuracy=tf.reduce_mean(tf.cast(prediction,tf.float32))

### Session

In [28]:
#  with tf.Session() as sess:
#     sess.run(tf.global_variables_initializer())
#     for j in range(500):
#         sess.run(training_init_op)        
#         print(sess.run([train_op,loss_op,accuracy]))
#         sess.run(validation_init_op)
#         print('Validation Batch Accuracy',j,sess.run(accuracy))

In [29]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(training_init_op)
    it = 0
    while True:
        
        try:
            it += 1
            #sess.run([X,y])
            print(sess.run([train_op,loss_op,accuracy]))
            print(it)
        except tf.errors.OutOfRangeError:
            print ('end of dataset')
            break
    sess.run(validation_init_op)
    it = 0
    while True:
        
        try:
            it += 1
            #sess.run([X,y])
            print(sess.run(accuracy))
            print(it)
        except tf.errors.OutOfRangeError:
            print ('end of dataset')
            break

[None, 0.7354389, 0.375]
1
[None, 49.65248, 0.5]
2
[None, 36.58546, 0.6875]
3
[None, 1.779691, 0.5]
4
[None, 51.016098, 0.625]
5
[None, 49.957817, 0.5625]
6
[None, 14.565824, 0.5625]
7
[None, 16.877853, 0.625]
8
[None, 39.6596, 0.5]
9
[None, 7.09534, 0.75]
10
[None, 14.416508, 0.5]
11
[None, 34.61103, 0.375]
12
[None, 15.646998, 0.5]
13
[None, 3.329422, 0.4375]
14
[None, 7.269452, 0.6875]
15
[None, 11.728121, 0.6875]
16
[None, 12.227739, 0.625]
17
[None, 12.792631, 0.4375]
18
[None, 2.3575664, 0.4375]
19
[None, 11.008345, 0.5]
20
[None, 8.125898, 0.625]
21
[None, 6.1957893, 0.75]
22
[None, 6.2559958, 0.5]
23
[None, 1.5394294, 0.4375]
24
[None, 1.6993039, 0.75]
25
[None, 5.4371843, 0.5625]
26
[None, 3.7410977, 0.6875]
27
[None, 6.3667035, 0.375]
28
[None, 3.2052875, 0.375]
29
[None, 1.921778, 0.5]
30
[None, 3.0734446, 0.5]
31
[None, 2.9994535, 0.625]
32
[None, 3.47609, 0.5]
33
[None, 2.6322777, 0.4375]
34
[None, 0.9051722, 0.625]
35
[None, 0.8901596, 0.75]
36
[None, 2.3028753, 0.5]
37
[

[None, 0.7478636, 0.4375]
285
[None, 0.5278549, 0.6875]
286
[None, 0.6130831, 0.6875]
287
[None, 0.7323396, 0.5]
288
[None, 0.72686803, 0.5]
289
[None, 0.5313562, 0.75]
290
[None, 0.6744544, 0.625]
291
[None, 0.6217888, 0.75]
292
[None, 0.6621736, 0.6875]
293
[None, 0.5644706, 0.6875]
294
[None, 0.7329142, 0.625]
295
[None, 0.67309237, 0.625]
296
[None, 0.71906304, 0.625]
297
[None, 0.6011605, 0.6875]
298
[None, 0.55575895, 0.5625]
299
[None, 0.6295352, 0.6875]
300
[None, 0.5662779, 0.75]
301
[None, 0.8931967, 0.4375]
302
[None, 0.83818305, 0.5]
303
[None, 0.7561293, 0.4375]
304
[None, 0.6294699, 0.5625]
305
[None, 0.6602468, 0.6875]
306
[None, 0.6714568, 0.5]
307
[None, 0.58429646, 0.6875]
308
[None, 0.61110735, 0.625]
309
[None, 0.68173754, 0.625]
310
[None, 0.57132137, 0.75]
311
[None, 0.51457036, 0.6875]
312
[None, 0.59255826, 0.625]
313
[None, 0.7568369, 0.625]
314
[None, 0.5997689, 0.625]
315
[None, 0.59280115, 0.5625]
316
[None, 0.5473387, 0.75]
317
[None, 0.5933405, 0.75]
318
[

[None, 0.58429825, 0.6875]
563
[None, 0.61057615, 0.6875]
564
[None, 0.610844, 0.625]
565
[None, 0.41737545, 0.9375]
566
[None, 0.66959727, 0.5625]
567
[None, 0.44415113, 0.8125]
568
[None, 0.53773767, 0.75]
569
[None, 0.63812184, 0.625]
570
[None, 0.6779939, 0.625]
571
[None, 0.62588096, 0.625]
572
[None, 0.67738736, 0.5]
573
[None, 0.6670526, 0.5625]
574
[None, 0.5791653, 0.75]
575
[None, 0.772254, 0.375]
576
[None, 0.704515, 0.5]
577
[None, 0.6708926, 0.5]
578
[None, 0.67147386, 0.75]
579
[None, 0.6801579, 0.625]
580
[None, 0.6022093, 0.6875]
581
[None, 0.7023673, 0.5625]
582
[None, 0.6769763, 0.5625]
583
[None, 0.72642624, 0.625]
584
[None, 0.6990094, 0.5625]
585
[None, 0.69357985, 0.5]
586
[None, 0.6729538, 0.5]
587
[None, 0.5531331, 0.6875]
588
[None, 0.54369813, 0.8125]
589
[None, 0.5470539, 0.75]
590
[None, 0.58264005, 0.75]
591
[None, 0.62086666, 0.5625]
592
[None, 0.595818, 0.75]
593
[None, 0.6371091, 0.625]
594
[None, 0.6487638, 0.625]
595
[None, 0.5318053, 0.6875]
596
[None

[None, 0.77107424, 0.375]
841
[None, 0.6851506, 0.5625]
842
[None, 0.5773263, 0.75]
843
[None, 0.64204234, 0.625]
844
[None, 0.6330497, 0.875]
845
[None, 0.7410188, 0.375]
846
[None, 0.593656, 0.6875]
847
[None, 0.6572328, 0.5625]
848
[None, 0.6464733, 0.6875]
849
[None, 0.63155866, 0.5625]
850
[None, 0.68131554, 0.625]
851
[None, 0.680446, 0.5625]
852
[None, 0.68655103, 0.5625]
853
[None, 0.59067035, 0.6875]
854
[None, 0.7695014, 0.375]
855
[None, 0.5305623, 0.8125]
856
[None, 0.6547526, 0.6875]
857
[None, 0.5728857, 0.8125]
858
[None, 0.78769267, 0.4375]
859
[None, 0.69151795, 0.625]
860
[None, 0.7336761, 0.375]
861
[None, 0.62282604, 0.75]
862
[None, 0.6986582, 0.5625]
863
[None, 0.62930256, 0.75]
864
[None, 0.62979823, 0.8125]
865
[None, 0.55371034, 0.8125]
866
[None, 0.6527699, 0.5]
867
[None, 0.6661835, 0.625]
868
[None, 0.68584913, 0.5625]
869
[None, 0.7711706, 0.5625]
870
[None, 0.62402385, 0.625]
871
[None, 0.564245, 0.6875]
872
[None, 0.5636337, 0.8125]
873
[None, 0.6394589, 

[None, 0.69067955, 0.5625]
1115
[None, 0.6954925, 0.4375]
1116
[None, 0.6375803, 0.625]
1117
[None, 0.6740974, 0.5]
1118
[None, 0.6907438, 0.5625]
1119
[None, 0.5648084, 0.6875]
1120
[None, 0.6034522, 0.8125]
1121
[None, 0.7988336, 0.6875]
1122
[None, 0.59523, 0.6875]
1123
[None, 0.57488024, 0.6875]
1124
[None, 0.4588439, 0.8125]
1125
[None, 0.56900704, 0.6875]
1126
[None, 0.6333064, 0.625]
1127
[None, 0.68542415, 0.5]
1128
[None, 0.48061174, 0.875]
1129
[None, 0.56654465, 0.75]
1130
[None, 0.63849634, 0.5625]
1131
[None, 0.5828962, 0.5625]
1132
[None, 0.6883903, 0.6875]
1133
[None, 0.73338157, 0.4375]
1134
[None, 0.651615, 0.625]
1135
[None, 0.64791626, 0.6875]
1136
[None, 0.63962716, 0.625]
1137
[None, 0.6051051, 0.625]
1138
[None, 0.64936435, 0.75]
1139
[None, 0.6086575, 0.5625]
1140
[None, 0.5941248, 0.625]
1141
[None, 0.6720815, 0.5]
1142
[None, 0.53020227, 0.75]
1143
[None, 0.5982311, 0.625]
1144
[None, 0.7760285, 0.375]
1145
[None, 0.4738865, 0.9375]
1146
[None, 0.5720465, 0.625

...

[None, 0.7013113, 0.625]
6219
[None, 0.5190867, 0.75]
6220
[None, 0.37901443, 0.8125]
6221
[None, 0.65300846, 0.625]
6222
[None, 0.54247355, 0.5625]
6223
[None, 0.6181433, 0.625]
6224
[None, 0.5748721, 0.75]
6225
[None, 0.5004211, 0.8125]
6226
[None, 0.36682394, 0.75]
6227
[None, 0.45731938, 0.75]
6228
[None, 0.53652924, 0.6875]
6229
[None, 0.637745, 0.75]
6230
[None, 0.7285762, 0.5625]
6231
[None, 0.4869873, 0.8125]
6232
[None, 0.6257689, 0.625]
6233
[None, 0.6939187, 0.5625]
6234
[None, 0.49979472, 0.5625]
6235
[None, 0.7399812, 0.6875]
6236
[None, 1.075789, 0.5625]
6237
[None, 0.55813175, 0.75]
6238
[None, 1.1035337, 0.4375]
6239
[None, 0.5216991, 0.625]
6240
[None, 0.8187854, 0.625]
6241
[None, 0.23115253, 0.9375]
6242
[None, 0.5873544, 0.8125]
6243
[None, 0.4079039, 0.875]
6244
[None, 0.58351535, 0.625]
6245
[None, 0.4726954, 0.6875]
6246
[None, 0.83781374, 0.5625]
6247
[None, 0.5507886, 0.625]
6248
[None, 0.8052713, 0.6875]
6249
[None, 0.6097723, 0.75]
6250
[None, 0.4124506, 0.68

[None, 0.7080212, 0.625]
6486
[None, 0.5068245, 0.6875]
6487
[None, 0.45010793, 0.6875]
6488
[None, 0.7404056, 0.5625]
6489
[None, 0.64065266, 0.6875]
6490
[None, 0.75822794, 0.4375]
6491
[None, 0.6802045, 0.5625]
6492
[None, 0.643651, 0.625]
6493
[None, 0.55276084, 0.75]
6494
[None, 0.55165553, 0.75]
6495
[None, 0.69620323, 0.6875]
6496
[None, 0.5006008, 0.625]
6497
[None, 0.6480037, 0.5]
6498
[None, 0.63825047, 0.5625]
6499
[None, 0.49841875, 0.6875]
6500
[None, 0.4981878, 0.625]
6501
[None, 0.7052598, 0.5]
6502
[None, 0.7093039, 0.625]
6503
[None, 0.6091751, 0.6875]
6504
[None, 0.4119411, 0.875]
6505
[None, 0.5778418, 0.8125]
6506
[None, 0.43229213, 0.8125]
6507
[None, 0.8040098, 0.625]
6508
[None, 0.6323106, 0.5625]
6509
[None, 0.389263, 0.875]
6510
[None, 0.7390813, 0.625]
6511
[None, 1.2169845, 0.5]
6512
[None, 0.41255605, 0.75]
6513
[None, 0.5014581, 0.75]
6514
[None, 0.99491763, 0.5]
6515
[None, 0.2963795, 0.9375]
6516
[None, 0.4973489, 0.8125]
6517
[None, 0.6085041, 0.625]
651

[None, 0.38196605, 0.75]
6754
[None, 0.5694862, 0.6875]
6755
[None, 0.6112534, 0.75]
6756
[None, 0.6306962, 0.625]
6757
[None, 0.8703972, 0.5625]
6758
[None, 0.70072305, 0.625]
6759
[None, 0.8355991, 0.625]
6760
[None, 0.3496341, 0.875]
6761
[None, 0.68681467, 0.6875]
6762
[None, 0.43704522, 0.75]
6763
[None, 0.5109478, 0.75]
6764
[None, 0.7999897, 0.5625]
6765
[None, 0.64298964, 0.6875]
6766
[None, 0.7267954, 0.6875]
6767
[None, 0.5173707, 0.8125]
6768
[None, 0.5287918, 0.625]
6769
[None, 0.8025186, 0.625]
6770
[None, 0.7981598, 0.5625]
6771
[None, 0.7042827, 0.5]
6772
[None, 0.572701, 0.625]
6773
[None, 0.6096645, 0.8125]
6774
[None, 0.5514394, 0.5625]
6775
[None, 0.48095056, 0.75]
6776
[None, 0.4119023, 0.75]
6777
[None, 0.85826117, 0.5]
6778
[None, 0.35794955, 0.8125]
6779
[None, 0.691804, 0.625]
6780
[None, 0.7233216, 0.6875]
6781
[None, 0.74861586, 0.5]
6782
[None, 0.6006659, 0.6875]
6783
[None, 0.455005, 0.8125]
6784
[None, 0.45984378, 0.8125]
6785
[None, 0.74528646, 0.75]
6786


[None, 0.6425697, 0.5]
7023
[None, 0.76842946, 0.625]
7024
[None, 0.5035508, 0.6875]
7025
[None, 0.64152145, 0.5]
7026
[None, 0.53010684, 0.6875]
7027
[None, 0.6053125, 0.75]
7028
[None, 0.49562114, 0.625]
7029
[None, 0.5602621, 0.75]
7030
[None, 0.641945, 0.6875]
7031
[None, 0.6300212, 0.5625]
7032
[None, 0.5919533, 0.75]
7033
[None, 0.7456466, 0.8125]
7034
[None, 0.5545629, 0.75]
7035
[None, 0.9377742, 0.5]
7036
[None, 0.56696427, 0.625]
7037
[None, 0.6035422, 0.8125]
7038
[None, 0.6312914, 0.6875]
7039
[None, 0.7597156, 0.5625]
7040
[None, 0.35368034, 0.8125]
7041
[None, 0.60762566, 0.6875]
7042
[None, 0.5876297, 0.625]
7043
[None, 0.33615986, 0.875]
7044
[None, 0.72102135, 0.5625]
7045
[None, 0.6323673, 0.5625]
7046
[None, 0.6339494, 0.625]
7047
[None, 0.7203958, 0.5]
7048
[None, 0.5705712, 0.6875]
7049
[None, 0.5473591, 0.75]
7050
[None, 0.6941341, 0.5]
7051
[None, 0.62696064, 0.625]
7052
[None, 0.556635, 0.5625]
7053
[None, 0.619275, 0.625]
7054
[None, 0.6939224, 0.75]
7055
[None

[None, 0.4866681, 0.6875]
7292
[None, 0.9634776, 0.5625]
7293
[None, 0.4616272, 0.75]
7294
[None, 0.7455207, 0.5]
7295
[None, 0.46573257, 0.875]
7296
[None, 0.8278093, 0.5625]
7297
[None, 0.74954474, 0.6875]
7298
[None, 0.45242408, 0.8125]
7299
[None, 0.58331996, 0.6875]
7300
[None, 0.6691464, 0.75]
7301
[None, 0.60792613, 0.6875]
7302
[None, 0.76585925, 0.5625]
7303
[None, 0.91840637, 0.4375]
7304
[None, 0.33543307, 0.875]
7305
[None, 0.52608097, 0.75]
7306
[None, 0.5675019, 0.6875]
7307
[None, 0.42378718, 0.875]
7308
[None, 0.5567449, 0.6875]
7309
[None, 0.8259786, 0.5]
7310
[None, 0.6345281, 0.5]
7311
[None, 0.5371952, 0.625]
7312
[None, 0.3663552, 0.875]
7313
[None, 0.5585225, 0.8125]
7314
[None, 0.467178, 0.75]
7315
[None, 0.49478096, 0.8125]
7316
[None, 0.67830276, 0.6875]
7317
[None, 0.63587993, 0.625]
7318
[None, 0.38263685, 0.8125]
7319
[None, 0.6362066, 0.625]
7320
[None, 0.5297, 0.75]
7321
[None, 0.4406073, 0.875]
7322
[None, 0.41955948, 0.8125]
7323
[None, 0.6457693, 0.75]
